# Importaciones

In [41]:
# Importar librerías necesarias
import os
import glob
import random
import pandas as pd

# Establecer la semilla para reproducibilidad
random.seed(42)

# Definir la carpeta para guardar los CSV
csv_dir = "./results/csv_files"
os.makedirs(csv_dir, exist_ok=True)  # Crea la carpeta si no existe

# Directorio raíz de las imágenes reales
# (Carpeta que contiene subcarpetas "Bacterial", "COVID-19" y "Normal")
real_data_dir = "./data/covid-chest-xray"

# Directorio de las imágenes aumentadas
# (Carpeta que contiene subcarpetas "Bacterial", "COVID-19" y "Normal")
augmented_data_dir = "./data/data_augmentation"

# Lista de clases según la estructura de tus carpetas
clases = ["Bacterial", "COVID-19", "Normal"]

In [42]:
def obtener_rutas_imagenes(ruta_base, extension=("jpg", "jpeg", "png", "tiff")):
    rutas = []
    for ext in extension:
        rutas.extend(glob.glob(os.path.join(ruta_base, f"*.{ext}")))
    return rutas


# Crear un diccionario con las rutas de imágenes reales para cada clase
imagenes_reales = {}
for clase in clases:
    ruta_clase = os.path.join(real_data_dir, clase)
    imagenes_reales[clase] = obtener_rutas_imagenes(ruta_clase)
    print(f"{clase}: {len(imagenes_reales[clase])} imágenes reales")

Bacterial: 48 imágenes reales
COVID-19: 342 imágenes reales
Normal: 486 imágenes reales


In [43]:
# Definir porcentajes
porc_train = 0.70
porc_val = 0.15
porc_test = 0.15

# Diccionarios para almacenar las rutas de cada conjunto
train_real = {}
val_real = {}
test_real = {}

for clase in clases:
    imagenes = imagenes_reales[clase].copy()
    random.shuffle(imagenes)

    total = len(imagenes)
    n_train = int(total * porc_train)
    n_val = int(total * porc_val)
    # El resto va a test, asegurando que se asignen todas las imágenes
    n_test = total - n_train - n_val

    train_real[clase] = imagenes[:n_train]
    val_real[clase] = imagenes[n_train : n_train + n_val]
    test_real[clase] = imagenes[n_train + n_val :]

    print(
        f"{clase} - Total: {total}, Train: {len(train_real[clase])}, Val: {len(val_real[clase])}, Test: {len(test_real[clase])}"
    )

Bacterial - Total: 48, Train: 33, Val: 7, Test: 8
COVID-19 - Total: 342, Train: 239, Val: 51, Test: 52
Normal - Total: 486, Train: 340, Val: 72, Test: 74


In [ ]:
def crear_registro(rutas_dict, conjunto_name):
    registros = []
    for clase, rutas in rutas_dict.items():
        for ruta in rutas:
            registros.append({"Conjunto": conjunto_name, "Clase": clase, "Ruta": ruta})
    return pd.DataFrame(registros)


df_train = crear_registro(train_real, "train")
df_val = crear_registro(val_real, "val")
df_test = crear_registro(test_real, "test")

# Concatenar y mostrar el registro completo
df_registro = pd.concat([df_train, df_val, df_test], ignore_index=True)
print(df_registro.head())
print(df_registro.groupby(["Conjunto", "Clase"]).size())

# Guardar el registro de imágenes reales
csv_registro_imagenes = os.path.join(csv_dir, "registro_imagenes_reales_multiclass.csv")
df_registro.to_csv(csv_registro_imagenes, index=False)
print(f"Registro de imágenes guardado en: {csv_registro_imagenes}")

  Conjunto      Clase                                               Ruta
0    train  Bacterial  ./data/covid-chest-xray\Bacterial\Bacterial-45...
1    train  Bacterial  ./data/covid-chest-xray\Bacterial\Bacterial-4.jpg
2    train  Bacterial  ./data/covid-chest-xray\Bacterial\Bacterial-40...
3    train  Bacterial  ./data/covid-chest-xray\Bacterial\Bacterial-2.jpg
4    train  Bacterial  ./data/covid-chest-xray\Bacterial\Bacterial-38...
Conjunto  Clase    
test      Bacterial      8
          COVID-19      52
          Normal        74
train     Bacterial     33
          COVID-19     239
          Normal       340
val       Bacterial      7
          COVID-19      51
          Normal        72
dtype: int64
Registro de imágenes guardado en: ./results/csv_files\registro_imagenes_reales.csv


In [45]:
# Diccionario para almacenar rutas de imágenes aumentadas por clase
imagenes_aug = {}
for clase in clases:
    ruta_clase_aug = os.path.join(augmented_data_dir, clase)
    imagenes_aug[clase] = obtener_rutas_imagenes(ruta_clase_aug)
    print(f"{clase}: {len(imagenes_aug[clase])} imágenes aumentadas")

Bacterial: 1000 imágenes aumentadas
COVID-19: 1000 imágenes aumentadas
Normal: 1000 imágenes aumentadas


In [46]:
# Diccionario para el conjunto de entrenamiento final (reales + aumentadas)
train_final = {}

for clase in clases:
    reales = train_real[clase]
    aumentadas = imagenes_aug.get(clase, [])
    train_final[clase] = reales + aumentadas
    print(
        f"{clase} - Train final: {len(reales)} reales + {len(aumentadas)} aumentadas = {len(train_final[clase])} imágenes"
    )

Bacterial - Train final: 33 reales + 1000 aumentadas = 1033 imágenes
COVID-19 - Train final: 239 reales + 1000 aumentadas = 1239 imágenes
Normal - Train final: 340 reales + 1000 aumentadas = 1340 imágenes


In [ ]:
df_train_final = crear_registro(train_final, "train_final")

# Mostrar resumen por clase
print(df_train_final.groupby("Clase").size())

# Guardar el registro del conjunto de entrenamiento final (reales + aumentadas)
csv_train_final = os.path.join(csv_dir, "registro_train_final_multiclass.csv")
df_train_final.to_csv(csv_train_final, index=False)
print(f"Registro del conjunto de entrenamiento final guardado en: {csv_train_final}")

Clase
Bacterial    1033
COVID-19     1239
Normal       1340
dtype: int64
Registro del conjunto de entrenamiento final guardado en: ./results/csv_files\registro_train_final.csv


In [48]:
# Resumen de las cantidades en cada conjunto
resumen = {
    "Conjunto": ["Train (reales)", "Val", "Test"],
    "Bacterial": [
        len(train_real["Bacterial"]),
        len(val_real["Bacterial"]),
        len(test_real["Bacterial"]),
    ],
    "COVID-19": [
        len(train_real["COVID-19"]),
        len(val_real["COVID-19"]),
        len(test_real["COVID-19"]),
    ],
    "Normal": [
        len(train_real["Normal"]),
        len(val_real["Normal"]),
        len(test_real["Normal"]),
    ],
}

df_resumen = pd.DataFrame(resumen)
print("División de imágenes reales:")
print(df_resumen)

# Resumen de imágenes en el entrenamiento final (reales + aumentadas)
resumen_train_final = {
    "Clase": clases,  # ["Bacterial", "COVID-19", "Normal"]
    "Train_final": [len(train_final[clase]) for clase in clases],
}

df_resumen_train_final = pd.DataFrame(resumen_train_final)
print("\nConjunto de entrenamiento final (reales + aumentadas):")
print(df_resumen_train_final)

División de imágenes reales:
         Conjunto  Bacterial  COVID-19  Normal
0  Train (reales)         33       239     340
1             Val          7        51      72
2            Test          8        52      74

Conjunto de entrenamiento final (reales + aumentadas):
       Clase  Train_final
0  Bacterial         1033
1   COVID-19         1239
2     Normal         1340


In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Crear generador para el conjunto de entrenamiento (solo rescaling)
train_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Crear el generador de datos a partir del DataFrame de entrenamiento (imágenes reales + aumentadas)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train_final,  # DataFrame con las imágenes de entrenamiento
    x_col="Ruta",  # Columna con las rutas de las imágenes
    y_col="Clase",  # Columna con las etiquetas de clase
    target_size=(224, 224),  # Redimensionar las imágenes a 224x224
    batch_size=32,
    class_mode="categorical",  # Para clasificación multiclase
    shuffle=True,
)

print("Clases encontradas:", train_generator.class_indices)

# Crear generador para test (solo rescaling)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Filtrar el DataFrame para obtener solo las imágenes del conjunto "test"
df_test = df_registro[df_registro["Conjunto"] == "test"]

# Crear el generador de datos para el conjunto de test
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,  # DataFrame con las imágenes reales de test
    x_col="Ruta",  # Columna que contiene las rutas de las imágenes
    y_col="Clase",  # Columna con las etiquetas de clase
    target_size=(224, 224),  # Redimensiona las imágenes a 224x224
    batch_size=32,
    class_mode="categorical",  # Para clasificación multiclase
    shuffle=False,  # No se baraja para mantener el orden de test
)

print("Clases encontradas en test:", test_generator.class_indices)

Found 3612 validated image filenames belonging to 3 classes.
Clases encontradas: {'Bacterial': 0, 'COVID-19': 1, 'Normal': 2}
Found 134 validated image filenames belonging to 3 classes.
Clases encontradas en test: {'Bacterial': 0, 'COVID-19': 1, 'Normal': 2}


In [50]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Dropout,
    Flatten,
    Dense,
    BatchNormalization,
    Activation,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
import json

# Definición del modelo

In [51]:
# =============================================================================
# Definir la función que construye el modelo de CNN, con hiperparámetros ajustables.
# =============================================================================
def build_cnn(hp):
    model = Sequential(
        [
            # Primera capa convolucional: se parametriza el número de filtros (entre 32 y 64, de 16 en 16)
            Conv2D(
                hp.Int("filters_1", 32, 64, step=16),
                (3, 3),
                activation=None,
                input_shape=(224, 224, 3),
            ),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling2D(2, 2),
            Dropout(hp.Float("dropout_1", 0.2, 0.5, step=0.1)),
            
            # Segunda capa convolucional
            Conv2D(hp.Int("filters_2", 64, 128, step=32), (3, 3), activation=None),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling2D(2, 2),
            Dropout(hp.Float("dropout_2", 0.2, 0.5, step=0.1)),
            
            # Tercera capa convolucional
            Conv2D(hp.Int("filters_3", 128, 256, step=64), (3, 3), activation=None),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling2D(2, 2),
            Dropout(hp.Float("dropout_3", 0.3, 0.6, step=0.1)),
            
            # Capa de aplanamiento para pasar a las capas densas
            Flatten(),
            
            # Capa densa con unidades parametrizables (entre 64 y 256, de 64 en 64)
            Dense(hp.Int("dense_units", 64, 256, step=64), activation=None),
            BatchNormalization(),
            Activation("relu"),
            Dropout(hp.Float("dropout_dense", 0.3, 0.6, step=0.1)),
            
            # Capa de salida: 3 neuronas para 3 clases, con activación softmax
            Dense(3, activation="softmax"),
        ]
    )

    # Definir el optimizador: Adam con tasa de aprendizaje seleccionada entre tres opciones.
    optimizer = Adam(learning_rate=hp.Choice("learning_rate", [1e-4, 5e-4, 1e-3]))

    # Compilar el modelo usando entropía cruzada categórica (para clasificación multiclase)
    # Se añaden varias métricas para evaluar: accuracy, precision, recall y AUC.
    model.compile(
        loss="categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy", "precision", "recall", "AUC"],
    )

    return model

# Carga de datos

# Entrenamiento del modelo

In [52]:
# =============================================================================
# Configurar el Random Search para buscar los mejores hiperparámetros
# =============================================================================
tuner = RandomSearch(
    build_cnn,  # Función que construye el modelo
    objective="val_accuracy",  # Métrica a optimizar (precisión en validación)
    max_trials=50,  # Número máximo de combinaciones a probar
    executions_per_trial=1,  # Número de ejecuciones por cada combinación
    directory="cnn_chest-xray_problem2",  # Directorio para guardar resultados del tuner
    project_name="cnn_chest-xray_problem2",  # Nombre del proyecto
)

# =============================================================================
# Definir Early Stopping para detener el entrenamiento si no mejora la precisión de validación
# =============================================================================
early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=5, restore_best_weights=True
)

# =============================================================================
# Realizar la búsqueda de hiperparámetros usando los generadores de entrenamiento y validación.
# Se asume que 'train_generator' y 'test_generator' están definidos previamente.
# =============================================================================
tuner.search(
    train_generator,
    validation_data=test_generator,  # Aquí test_generator actúa como conjunto de validación
    epochs=25,
    callbacks=[early_stopping],
    verbose=1,
)

Trial 9 Complete [00h 59m 27s]
val_accuracy: 0.9776119589805603

Best val_accuracy So Far: 0.9776119589805603
Total elapsed time: 07h 25m 11s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
64                |48                |filters_1
0.3               |0.3               |dropout_1
128               |96                |filters_2
0.4               |0.4               |dropout_2
128               |192               |filters_3
0.4               |0.5               |dropout_3
64                |128               |dense_units
0.4               |0.4               |dropout_dense
0.0005            |0.0005            |learning_rate

Epoch 1/25
113/113 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - AUC: 0.8498 - accuracy: 0.6695 - loss: 0.7881 - precision: 0.6971 - recall: 0.6282 - val_AUC: 0.4796 - val_accuracy: 0.3881 - val_loss: 2.2259 - val_precision: 0.3881 - val_recall: 0.3881
Epoch 2/25
113/113 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - AUC: 0.9299 - accuracy: 0.7857 - los

KeyboardInterrupt: 

In [53]:
# Mostrar resumen de la búsqueda de hiperparámetros
tuner.results_summary()

Results summary
Results in cnn_chest-xray_problem2\cnn_chest-xray_problem2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
filters_1: 48
dropout_1: 0.30000000000000004
filters_2: 96
dropout_2: 0.4
filters_3: 192
dropout_3: 0.5
dense_units: 128
dropout_dense: 0.4
learning_rate: 0.0005
Score: 0.9776119589805603

Trial 01 summary
Hyperparameters:
filters_1: 32
dropout_1: 0.2
filters_2: 128
dropout_2: 0.4
filters_3: 256
dropout_3: 0.3
dense_units: 192
dropout_dense: 0.4
learning_rate: 0.0005
Score: 0.9701492786407471

Trial 02 summary
Hyperparameters:
filters_1: 64
dropout_1: 0.30000000000000004
filters_2: 64
dropout_2: 0.2
filters_3: 256
dropout_3: 0.3
dense_units: 192
dropout_dense: 0.3
learning_rate: 0.0005
Score: 0.9701492786407471

Trial 06 summary
Hyperparameters:
filters_1: 48
dropout_1: 0.4
filters_2: 64
dropout_2: 0.2
filters_3: 128
dropout_3: 0.4
dense_units: 128
dropout_dense: 0.3
learning_rate: 0.0001
Score: 0.9701492786

In [54]:
# =============================================================================
# Obtener el mejor modelo y sus hiperparámetros encontrados
# =============================================================================
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]
print(
    f"Mejor modelo encontrado con filtros: {best_hps.get('filters_1')}, {best_hps.get('filters_2')}, {best_hps.get('filters_3')}, learning rate: {best_hps.get('learning_rate')}"
)

Mejor modelo encontrado con filtros: 48, 96, 192, learning rate: 0.0005


c:\Users\juanm\Desktop\Workspace\ia\AI-CNN-Pneumonia-COVID-Detection\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [55]:
# =============================================================================
# Entrenar el mejor modelo utilizando el conjunto de entrenamiento y validación.
# Se extrae el historial de entrenamiento para su análisis posterior.
# =============================================================================
history = best_model.fit(train_generator, validation_data=test_generator, epochs=25)

# =============================================================================
# Convertir el historial de entrenamiento a un DataFrame para analizar las métricas.
# =============================================================================
df_metrics = pd.DataFrame(history.history)

Epoch 1/25
 13/113 ━━━━━━━━━━━━━━━━━━━━ 2:33 2s/step - AUC: 0.9997 - accuracy: 0.9840 - loss: 0.0471 - precision: 0.9842 - recall: 0.9840

KeyboardInterrupt: 

In [ ]:
# =============================================================================
# Guardar el DataFrame con las métricas de entrenamiento en un archivo CSV,
# usando una estructura de carpetas organizada.
# =============================================================================
csv_dir = "./results/csv_files"
os.makedirs(csv_dir, exist_ok=True)

csv_metrics_path = os.path.join(csv_dir, "training_metrics_multiclass.csv")
df_metrics.to_csv(csv_metrics_path, index=False)
print("Métricas guardadas en:", csv_metrics_path)

NameError: name 'df_metrics' is not defined

In [ ]:
def save_model_structure(best_model, best_hps, model_type):
    """
    Guarda el modelo completo, sus pesos y los hiperparámetros en una estructura organizada.

    Args:
        best_model: Modelo de Keras a guardar.
        best_hps: Objeto de hiperparámetros (por ejemplo, de Keras Tuner).
        model_type: Cadena identificadora ("multiclass" o "binary") para crear subcarpetas específicas.
    """
    # Definir la carpeta base para el tipo de modelo (multiclass o binary)
    base_dir = f"./results/{model_type}"

    # Crear subcarpetas para modelos, pesos e hiperparámetros
    models_dir = os.path.join(base_dir, "models")
    weights_dir = os.path.join(base_dir, "weights")
    hyperparams_dir = os.path.join(base_dir, "hyperparameters")

    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(weights_dir, exist_ok=True)
    os.makedirs(hyperparams_dir, exist_ok=True)

    # Guardar el modelo completo en la carpeta 'models'
    model_path = os.path.join(models_dir, f"best_{model_type}_model.h5")
    best_model.save(model_path)
    print(f"Mejor {model_type} modelo guardado en:", model_path)

    # Guardar solo los pesos del modelo en la carpeta 'weights'
    weights_path = os.path.join(weights_dir, f"best_{model_type}.weights.h5")
    best_model.save_weights(weights_path)
    print(f"Pesos del {model_type} modelo guardados en:", weights_path)

    # Guardar los hiperparámetros del modelo en formato JSON en la carpeta 'hyperparameters'
    hyperparams_path = os.path.join(
        hyperparams_dir, f"best_{model_type}_hyperparameters.json"
    )
    with open(hyperparams_path, "w") as json_file:
        json.dump(best_hps.values, json_file, indent=4)
    print(f"Hiperparámetros del {model_type} modelo guardados en:", hyperparams_path)


# Ejemplo de uso para el modelo multiclase:
save_model_structure(best_model, best_hps, "multiclass")

Mejor multiclass modelo guardado en: ./results/multiclass\models\best_multiclass_model.h5


ValueError: The filename must end in `.weights.h5`. Received: filepath=./results/multiclass\weights\best_multiclass_weights.h5